In [2]:
import cv2
import mediapipe as mp


In [3]:
# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils


In [4]:
def count_fingers(landmarks):
    # Landmarks for finger tips
    tips_ids = [4, 8, 12, 16, 20]
    
    # Check if each finger tip landmark is above the base landmark
    fingers = []
    for i in range(len(tips_ids)):
        tip_y = landmarks[tips_ids[i]][1]
        base_y = landmarks[tips_ids[i] - 2][1] if i > 0 else landmarks[tips_ids[i] - 1][1]
        if tip_y < base_y:
            fingers.append(1)
        else:
            fingers.append(0)
    
    return sum(fingers)


In [5]:
def detect_gesture(num_fingers):
    gestures = {
        0: "Closed Fist",     # No fingers extended
        1: "Thumbs Up",       # 1 finger extended (thumb)
        2: "Peace Sign",      # 2 fingers extended (index and middle)
        3: "Three Fingers",   # 3 fingers extended (index, middle, ring)
        4: "Four Fingers",    # 4 fingers extended (all except thumb)
        5: "Open Hand"        # 5 fingers extended (open hand)
    }
    return gestures.get(num_fingers, "Unknown Gesture")


In [ ]:
# Capture video from webcam
cap = cv2.VideoCapture(0)

# Main loop to process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        break

    # Flip frame for a mirror effect and convert to RGB
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(frame_rgb)

    # Check if hands are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract landmark positions as (x, y) pairs
            landmarks = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

            # Count fingers
            num_fingers = count_fingers(landmarks)

            # Detect gesture based on number of fingers
            gesture_name = detect_gesture(num_fingers)

            # Display the detected gesture name
            cv2.putText(frame, gesture_name, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Hand Gesture Recognition - Press q to Quit', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
hands.close()
